In [1]:
import mxnet as mx
import numpy as np

In [2]:
a = mx.sym.Variable("a")
b = mx.sym.Variable("b")
c = a + b
print a.name, b.name
print type(c)

a b
<class 'mxnet.symbol.Symbol'>


Most NDarray operators can be applied to Symbol. Symbol describes the operation and procedure without the need to realize the values.
### note: dot product is an operator specified in symbol class, those in mx.nd. and numpy won't work. 

In [3]:
d = a * b
e = mx.sym.dot(a, b)
f = mx.sym.Reshape(a, shape=(2, 3)) # Reshape is capitalized. 
g = mx.sym.broadcast_to(f, shape=(2, 2, 3))

#### Symbol can be used to construct neural network layers.
One can use graphviz module to visualize the network in jupyter notebook. However, it doesn't work in Pycharm.

In [5]:
net = mx.sym.Variable("data")
net = mx.sym.FullyConnected(data=net, name="fc1", num_hidden=100)
net = mx.sym.Activation(data=net, name="relu1", act_type="relu")
net = mx.sym.FullyConnected(data=net, name="fc2", num_hidden=10)
net = mx.sym.SoftmaxOutput(data=net, name="out")

#### Examples about how Symbol is used to construct neural network is a little confusing.
declarative VS. imperative

#### Symbol manipulation

In [6]:
import mxnet as mx
a = mx.sym.Variable('a')
b = mx.sym.Variable('b')
c = a + b
assert a.name == "a", "Symbol name incorrect."
assert b.name == "b", "Symbol name incorrect."
(a, b, c)

(<mxnet.symbol.Symbol at 0x7f44bf345150>,
 <mxnet.symbol.Symbol at 0x7f44bf3fd090>)

In [12]:
d = mx.sym.dot(a, b)
arg_name = c.list_arguments()  # get the names of the inputs
out_name = c.list_outputs()    # get the names of the outputs
arg_shape, out_shape, _ = c.infer_shape(a=(3,3), b=(3,3))
print arg_name, arg_shape
print out_name, out_shape
print d.infer_shape(a=(3, 4), b=(4, 5))

['a', 'b'] [(3L, 3L), (3L, 3L)]
['_plus1_output'] [(3L, 3L)]
([(3L, 4L), (4L, 5L)], [(3L, 5L)], [])


In [38]:
a = mx.sym.Variable('label_a') # the label in Variable declaration has to match with that in the executor
b = mx.sym.Variable('label_b')
e = mx.sym.Variable("")
d = mx.sym.dot(a, b) + e
ex = d.bind(ctx=mx.cpu(), args={"label_b": mx.nd.array(np.ones((4, 4))),
                                "label_a": mx.nd.array(np.identity(4)),
                                "": mx.nd.array(np.zeros((4, 4))+1)})
ex.forward()
ex.outputs[0].asnumpy()

array([[ 2.,  2.,  2.,  2.],
       [ 2.,  2.,  2.,  2.],
       [ 2.,  2.,  2.,  2.],
       [ 2.,  2.,  2.,  2.]], dtype=float32)

In [45]:
# print(d.tojson())
d.save("symbol_d.json")
ddd = mx.sym.load("symbol_d.json")
ex = ddd.bind(ctx=mx.cpu(), args={"label_b": mx.nd.array(np.ones((4, 4))),
                                  "label_a": mx.nd.array(np.identity(4)),
                                  "": mx.nd.array(np.zeros((4, 4))+1)})
ex.outputs[0].asnumpy()

array([[ -9.65896516e+36,   4.56543040e-41,   1.28796234e-37,
          0.00000000e+00],
       [ -5.09295665e+29,   4.56543040e-41,  -3.36075876e+37,
          4.56543040e-41],
       [ -8.99095053e+33,   4.56543040e-41,  -1.93268863e+35,
          4.56543040e-41],
       [  3.82273773e-38,   0.00000000e+00,  -5.09292643e+29,
          4.56543040e-41]], dtype=float32)

### auto differentiation of Symbol